In [2]:
#Optional step to delete session if you get an error that multiple sessions are open
del session

# Initialize Notebook, import libraries and create Snowflake connection

In [3]:
from snowflake.snowpark.session import Session
from snowflake.snowpark.types import IntegerType, FloatType
from snowflake.snowpark.functions import avg, sum, col, udf, call_udf, call_builtin, year
import streamlit as st
import pandas as pd
from datetime import date

# scikit-learn (install: pip install -U scikit-learn)
from sklearn.linear_model import LinearRegression

# Session
# connection_parameters = {
#    "account": "<account_identifier>",
#    "user": "<username>",
#    "password": "<password>",
#    "warehouse": "compute_wh",
#    "role": "accountadmin",
#    "database": "summit_hol",
#    "schema": "public"
# }

import json 
with open(".env/creds.json") as f:
   connection_parameters = json.load(f)

connection_parameters['role'] = "ACCOUNTADMIN"
connection_parameters['database'] = "SUMMIT_HOL"
connection_parameters['schema'] = "PUBLIC"

session = Session.builder.configs(connection_parameters).create()



# test if we have a connection
session.sql("select current_warehouse() wh, current_database() db, current_schema() schema, current_version() v").show()


2023-02-16 10:38:17.090 INFO    snowflake.connector.connection: Snowflake Connector for Python Version: 2.7.12, Python Version: 3.8.15, Platform: macOS-10.16-x86_64-i386-64bit
2023-02-16 10:38:17.090 INFO    snowflake.connector.connection: This connection is in OCSP Fail Open Mode. TLS Certificates would be checked for validity and revocation status. Any other Certificate Revocation related exceptions or OCSP Responder failures would be disregarded in favor of connectivity.
2023-02-16 10:38:24.200 INFO    snowflake.snowpark.session: Snowpark Session information: 
"version" : 1.0.0,
"python.version" : 3.8.15,
"python.connector.version" : 2.7.12,
"python.connector.session.id" : 647878641406102,
"os.name" : Darwin

2023-02-16 10:38:24.203 INFO    snowflake.connector.cursor: query: [select current_warehouse() wh, current_database() db, current_schema() schema, c...]
2023-02-16 10:38:24.291 INFO    snowflake.connector.cursor: query execution done
2023-02-16 10:38:24.292 INFO    snowflake.co

------------------------------------------
|"WH"    |"DB"        |"SCHEMA"  |"V"    |
------------------------------------------
|XLARGE  |SUMMIT_HOL  |PUBLIC    |7.5.1  |
------------------------------------------



# Query the data

In [4]:
# SQL query to explore the data
session.sql("""SELECT * 
             FROM 
                ECONOMY_DATA_ATLAS.ECONOMY.BEANIPA 
            WHERE 
                TRUE
                AND "Table Name" = 'Price Indexes For Personal Consumption Expenditures By Major Type Of Product' 
                AND "Indicator Name" = 'Personal consumption expenditures (PCE)' AND "Frequency" = 'A' 
                ORDER BY "Date"
             """).show()

2023-02-16 10:38:30.410 INFO    snowflake.connector.cursor: query: [SELECT * FROM ECONOMY_DATA_ATLAS.ECONOMY.BEANIPA WHERE TRUE AND "Table Name" = '...]
2023-02-16 10:38:30.590 INFO    snowflake.connector.cursor: query execution done
2023-02-16 10:38:30.592 INFO    snowflake.connector.cursor: query: [SELECT  *  FROM (SELECT * FROM ECONOMY_DATA_ATLAS.ECONOMY.BEANIPA WHERE TRUE AND...]
2023-02-16 10:38:33.385 INFO    snowflake.connector.cursor: query execution done


----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"Table"  |"Table Name"                                        |"Table Description"  |"Table Full Name"                                   |"Table Unit"     |"Indicator"  |"Indicator Name"                         |"Indicator Description"  |"Indicator Full Name"  |"Units"          |"Scale"  |"Frequency"  |"Date"      |"Value"  |
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|T20304   |Pr

In [5]:
# Now use Snowpark dataframe
snow_df_pce = (session.table("ECONOMY_DATA_ATLAS.ECONOMY.BEANIPA") 
                            .filter(col('Table Name') == 'Price Indexes For Personal Consumption Expenditures By Major Type Of Product') 
                            .filter(col('Indicator Name') == 'Personal consumption expenditures (PCE)')
                            .filter(col('"Frequency"') == 'A')
                            .filter(col('"Date"') >= '1972-01-01'))
snow_df_pce.show()

2023-02-16 10:38:38.425 INFO    snowflake.connector.cursor: query: [SELECT  *  FROM ECONOMY_DATA_ATLAS.ECONOMY.BEANIPA]
2023-02-16 10:38:38.566 INFO    snowflake.connector.cursor: query execution done
2023-02-16 10:38:38.568 INFO    snowflake.connector.cursor: query: [SELECT  *  FROM ECONOMY_DATA_ATLAS.ECONOMY.BEANIPA WHERE (((("Table Name" = 'Pri...]
2023-02-16 10:38:39.819 INFO    snowflake.connector.cursor: query execution done


----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"Table"  |"Table Name"                                        |"Table Description"  |"Table Full Name"                                   |"Table Unit"     |"Indicator"  |"Indicator Name"                         |"Indicator Description"  |"Indicator Full Name"  |"Units"          |"Scale"  |"Frequency"  |"Date"      |"Value"  |
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|T20304   |Pr

In [6]:
# Let Snowflake perform filtering using the Snowpark pushdown and display results in a Pandas dataframe
snow_df_pce = (session.table("ECONOMY_DATA_ATLAS.ECONOMY.BEANIPA")
                        .filter(col('"Table Name"') == 'Price Indexes For Personal Consumption Expenditures By Major Type Of Product')
                        .filter(col('"Indicator Name"') == 'Personal consumption expenditures (PCE)')
                        .filter(col('"Frequency"') == 'A')
                        .filter(col('"Date"') >= '1972-01-01'))
pd_df_pce_year = snow_df_pce.select(year(col('"Date"')).alias('"Year"'), col('"Value"').alias('PCE') ).to_pandas()
pd_df_pce_year


2023-02-16 10:38:51.452 INFO    snowflake.connector.cursor: query: [SELECT  *  FROM ECONOMY_DATA_ATLAS.ECONOMY.BEANIPA]
2023-02-16 10:38:51.570 INFO    snowflake.connector.cursor: query execution done
2023-02-16 10:38:51.572 INFO    snowflake.connector.cursor: query: [SELECT year("Date") AS "Year", "Value" AS "PCE" FROM ( SELECT  *  FROM ECONOMY_D...]
2023-02-16 10:38:52.212 INFO    snowflake.connector.cursor: query execution done


Year      PCE
0   1972   22.542
1   1973   23.756
2   1974   26.229
3   1975   28.415
4   1976   29.974
5   1977   31.923
6   1978   34.145
7   1979   37.178
8   1980   41.182
9   1981   44.871
10  1982   47.363
11  1983   49.378
12  1984   51.243
13  1985   53.031
14  1986   54.184
15  1987   55.855
16  1988   58.038
17  1989   60.572
18  1990   63.231
19  1991   65.345
20  1992   67.087
21  1993   68.758
22  1994   70.193
23  1995   71.671
24  1996   73.204
25  1997   74.478
26  1998   75.070
27  1999   76.164
28  2000   78.090
29  2001   79.656
30  2002   80.702
31  2003   82.398
32  2004   84.443
33  2005   86.876
34  2006   89.322
35  2007   91.614
36  2008   94.325
37  2009   94.062
38  2010   95.747
39  2011   98.170
40  2012  100.000
41  2013  101.354
42  2014  102.887
43  2015  103.116
44  2016  104.148
45  2017  106.054
46  2018  108.317
47  2019  109.933
48  2020  111.145
49  2021  115.621
50  2022  122.817

# Train the Linear Regression model

In [7]:
# train model with PCE index

x = pd_df_pce_year["Year"].to_numpy().reshape(-1,1)
y = pd_df_pce_year["PCE"].to_numpy()

model = LinearRegression().fit(x, y)

# test model for 2022
predictYear = 2022
pce_pred = model.predict([[predictYear]])
# print the last 5 years
print (pd_df_pce_year.tail() )
# run the prediction for 2022
print ('Prediction for '+str(predictYear)+': '+ str(round(pce_pred[0],2)))


    Year      PCE
46  2018  108.317
47  2019  109.933
48  2020  111.145
49  2021  115.621
50  2022  122.817
Prediction for 2022: 118.37


### Creating a User Defined Function within Snowflake to do the scoring there

In [8]:
def predict_pce(predictYear: int) -> float:
    return model.predict([[predictYear]])[0].round(2).astype(float)

_ = session.udf.register(predict_pce,
                        return_type=FloatType(),
                        input_type=IntegerType(),
                        packages= ["pandas","scikit-learn"],
                        is_permanent=True, 
                        name="predict_pce_udf", 
                        replace=True,
                        stage_location="@udf_stage")

2023-02-16 10:39:44.325 INFO    snowflake.connector.cursor: query: [ls '@udf_stage']
2023-02-16 10:39:44.465 INFO    snowflake.connector.cursor: query execution done
2023-02-16 10:39:44.465 INFO    snowflake.connector.cursor: query: [SELECT "name" FROM ( SELECT  *  FROM  TABLE ( RESULT_SCAN('01aa5f67-0000-f8b3-00...]
2023-02-16 10:39:44.943 INFO    snowflake.connector.cursor: query execution done
2023-02-16 10:39:44.948 INFO    snowflake.connector.cursor: query: [SELECT  *  FROM information_schema.packages]
2023-02-16 10:39:45.062 INFO    snowflake.connector.cursor: query execution done
2023-02-16 10:39:45.067 INFO    snowflake.connector.cursor: query: [SELECT "PACKAGE_NAME", array_agg("VERSION") AS "ARRAY_AGG(VERSION)" FROM ( SELEC...]
2023-02-16 10:39:46.255 INFO    snowflake.connector.cursor: query execution done
2023-02-16 10:39:46.259 WARNING snowflake.snowpark.session: The version of package scikit-learn in the local environment is 1.2.0, which does not fit the criteria for the r

# Test the trained model by invoking the UDF via a SQL statement

In [9]:
session.sql("select predict_pce_udf(2022)").show()


2023-02-16 10:39:57.296 INFO    snowflake.connector.cursor: query: [select predict_pce_udf(2022)]
2023-02-16 10:39:57.403 INFO    snowflake.connector.cursor: query execution done
2023-02-16 10:39:57.404 INFO    snowflake.connector.cursor: query: [SELECT  *  FROM (select predict_pce_udf(2022)) LIMIT 10]
2023-02-16 10:40:03.699 INFO    snowflake.connector.cursor: query execution done


---------------------------
|"PREDICT_PCE_UDF(2022)"  |
---------------------------
|118.37                   |
---------------------------

